# Project

## Libraries

In [1]:
import requests
import json
import time
import pandas as pd
from tqdm import tqdm
import psycopg2
import yaml

# from pydantic import BaseModel, ValidationError, validator (## for the future)

/Users/yeungadrian/Documents/repo/ML-Ops/venv/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Riot Games API

In [2]:
with open("../config.yml", "r") as ymlfile:
    cfg = yaml.load(ymlfile)

/var/folders/8g/67y55mms29zf6yt6q92_3hsr0000gn/T/ipykernel_74366/1863735473.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(ymlfile)


In [9]:
api_key = cfg['riotgamesapi']['key']
api_header = {
    "X-Riot-Token": api_key
}

In [4]:
region = 'euw1'

In [5]:
challenger_url = f'https://{region}.api.riotgames.com/tft/league/v1/challenger?api_key={api_key}'
challenger_response = requests.get(url = challenger_url).json()

In [6]:
#Writes json response into a dataframe
challenger_response_df = pd.DataFrame(challenger_response['entries'])


In [7]:
challenger_response_df

,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,pQ0yVLGhdf0zmZ3fqJq_vYNRnGkj8wbmYzOdRbGcHAh84LUX,tftr1esok,842,I,38,220,False,False,False,False
1,jB6Is3aI6rWLwGznxJOJ_jWo_csRkNqXYj2t5v7Q2V03-BQ,20200603 lpl006,852,I,53,242,False,False,True,False
2,BBueamU-gTY463xq8mKeaEljzPGe_eu6do00OFG98Jt4le...,Gofret TFT,980,I,31,191,False,False,False,False
3,rPFMFDPZX7qUBha5wB6jEbaRxhGEZGUDiz_-0pFx7ObyVyE,NES Qetzer,1185,I,40,170,False,False,False,False
4,xJLsf7C13OrjkOBpbf9eDLYvVC4xZlhaJsGJ5f3H89djfj4,Binti,862,I,56,476,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...
195,f1A5Ca1jW32m0C0zzPkI3vmgnczNkT9hWJB1v-1IlgNDBto,huya GoDlike,887,I,56,250,False,False,True,False
196,jM1pT4Pozzs5k1ojgDblcmbo5M8lCsSqSa1WqwpHbmjrzJwN,JR70 Piechu,1129,I,48,327,True,False,False,False
197,PxGBPFGbeK3ZadtEk6WHoLlvk_bBnFyZB7gKESZcUmJnluyk,ARMA ACKK,1658,I,70,275,True,False,False,False
198,ApjyM70MczBu_we5EgTJt6_jShCOKz2sm0TT08chtogLG64,SeyRo,879,I,30,162,False,False,True,False


In [8]:
summonerId = challenger_response_df['summonerId'][10]
puuid_url = f'https://euw1.api.riotgames.com/tft/summoner/v1/summoners/{summonerId}'


In [13]:
puuid_from_summoner = requests.get(puuid_url, headers=api_header).json()

In [15]:
puuid = puuid_from_summoner['puuid']

In [23]:
matchid_url = f'https://europe.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?count=20'
matchids = requests.get(matchid_url, headers=api_header).json()

In [32]:
matchid = matchids[0]

In [33]:
match_info_url = f'https://europe.api.riotgames.com/tft/match/v1/matches/{matchid}'
match_info = requests.get(match_info_url, headers = api_header).json()

1. Call the api for 1 match
2. Insert into Match
3. Get the Id, you inserted
4. For the 8 players
    - Insert into MatchInfo, with Id from the Match Table
    - Get the id you inserted into the MatchInfo table
    - Insert into MatchTraits
    - Insert into MatchUnits

## Postgresql Database

In [4]:
params = cfg['postgresql']
# Connect to the PostgreSQL database
conn = psycopg2.connect(**params, port = 5432)
# Create a new cursor - A PostgreSQL database cursor is a read-only pointer that allows a program, regardless of the language used, to access the result set of a query. 
cur = conn.cursor()
# A function that takes in a PostgreSQL query and outputs a pandas database 

In [36]:
#Loops through rows for df
for i in tqdm(range(0, challenger_response_df.shape[0])):
    summonerId = challenger_response_df["summonerId"][i]
    # do all the columns that are called TBD

    cur.execute(f'''
    INSERT INTO public."Users"(
	    id, "summonerName", "summonerId", "rankedLeague", puuid, leaguepoints, region)
	    VALUES (DEFAULT, 'TBD', 'TBD', 'CHALLENGER', 'Temporary', null, {region})
        WHERE 
        ''')

    conn.commit()

KeyboardInterrupt: 

In [9]:
#Use this cell to delete table data, although ID counter needs to be reset manually in Postgres
cur.execute(f'''
DELETE FROM public."Users"
''')

In [12]:
conn.commit()

# Try...
#Incorrect, then rollback transaction

Commit or roll-back...

for loop:
take API and put into SQL 
Execute 1 person at a time...
Validation of data type...

psychopg to insert data into database

## Prefect

In [13]:
import prefect
from prefect import task, Flow

@task
def hello_task():
    logger = prefect.context.get("logger")
    logger.info("Hello world!")

with Flow("hello-flow") as flow:
    hello_task()

flow.run()

[2021-09-29 22:06:01+0100] INFO - prefect.FlowRunner | Beginning Flow run for 'hello-flow'
[2021-09-29 22:06:01+0100] INFO - prefect.TaskRunner | Task 'hello_task': Starting task run...
[2021-09-29 22:06:01+0100] INFO - prefect.hello_task | Hello world!
[2021-09-29 22:06:01+0100] INFO - prefect.TaskRunner | Task 'hello_task': Finished task run for task with final state: 'Success'
[2021-09-29 22:06:01+0100] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


<Success: "All reference tasks succeeded.">

In [14]:
from prefect import task

@task
def extract():
    """Get a list of data"""
    return [1, 2, 3]

@task
def transform(data):
    """Multiply the input by 10"""
    return [i * 10 for i in data]

@task
def load(data):
    """Print the data to indicate it was received"""
    print("Here's your data: {}".format(data))


In [20]:
from prefect import task, Flow, Parameter


@task(log_stdout=True)
def say_hello(name):
    print("Hello, {}!".format(name))


with Flow("My First Flow") as flow:
    name = Parameter('name')
    say_hello(name)


flow.run(name='world') # "Hello, world!"
flow.run(name='Marvin') # "Hello, Marvin!"

[2021-09-29 22:15:05+0100] INFO - prefect.FlowRunner | Beginning Flow run for 'My First Flow'
[2021-09-29 22:15:05+0100] INFO - prefect.TaskRunner | Task 'name': Starting task run...
[2021-09-29 22:15:05+0100] INFO - prefect.TaskRunner | Task 'name': Finished task run for task with final state: 'Success'
[2021-09-29 22:15:05+0100] INFO - prefect.TaskRunner | Task 'say_hello': Starting task run...
[2021-09-29 22:15:05+0100] INFO - prefect.TaskRunner | Hello, world!
[2021-09-29 22:15:05+0100] INFO - prefect.TaskRunner | Task 'say_hello': Finished task run for task with final state: 'Success'
[2021-09-29 22:15:05+0100] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2021-09-29 22:15:05+0100] INFO - prefect.FlowRunner | Beginning Flow run for 'My First Flow'
[2021-09-29 22:15:05+0100] INFO - prefect.TaskRunner | Task 'name': Starting task run...
[2021-09-29 22:15:05+0100] INFO - prefect.TaskRunner | Task 'name': Finished task run for task with final state: 'Suc

<Success: "All reference tasks succeeded.">